In [74]:
import nest_asyncio
import aiohttp
import asyncio
import nest_asyncio
import re
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from os import getenv
from openai import OpenAI
import pathlib
import textwrap
# import google.generativeai as genai
import time
import requests
import ast
import json
from sklearn.metrics.pairwise import cosine_similarity
import re
from llms import chatGPT
from llms import gemini


x_chat = chatGPT()
x_gemini = gemini()

In [119]:
df = pd.read_csv("../output/data/data_post_chat_gpt.csv",index_col=0)
# df = pd.read_csv("../output/data/2014-data.csv",index_col=0)
# df = df[df["headquarters location"].str.contains("United States")]
# df = df[pd.to_datetime(df['founded date'])<='11/30/2022']

In [120]:
df = df.sample(450, random_state=2).reset_index(drop=True)
#[df["organization name"].isin(["AI Mavericks","AI Teaches Me from BAM Software Development","AI Virtual Staging"])]
print("PRE", len(df[pd.to_datetime(df['founded date'])<='11/30/2022']))
print("POST", len(df[pd.to_datetime(df['founded date'])>='11/30/2022']))

PRE 0
POST 450


In [121]:
df2 = pd.read_csv("../output/data/2014-data.csv",index_col=0)
df = df[["organization name","website","description_all","founded date"]]
df2 = df2[["organization name","website","description_all","founded date"]]
df = pd.concat([df,df2]).reset_index(drop=True)
df.head()

,organization name,website,description_all,founded date
0,AI Mavericks,aimavericks.ai/about/,Event Management Company AI Mavericks is an ev...,2023-10-24
1,Campaign Brain,www.campaign-brain.com,Campaign Brain offers AI-driven tools for targ...,2023-01-01
2,Sentius,www.sentius.ai/,We automate enterprise tasks with Generative A...,2023-05-04
3,Vadar Alpha,vadar.ai,Vadar Alpha provides intelligence augmentation...,2024-01-01
4,Cognifyze,www.cognifyze.com,Our SaaS platform uses AI to analyze sales dat...,2023-10-06


In [123]:
df.tail()

,organization name,website,description_all,founded date
895,Mibio | Medical,www.mibio.io/,"Specialties: innovative analytics, data analyt...",2014-05-01
896,JNN GROUP,www.jnngroup.com/,Analytics + Risk & Consulting Based in Pasaden...,2014-04-07
897,Cloudstrats,www.cloudstrats.com,A cloud automation solutions and services comp...,2014-02-14
898,The Tuna Group,www.thetunagroup.com/,The Tuna Group is an IT firm that offers web d...,2014-01-01
899,Boray Data Technology,rapidsdb.com,Boray Data Technology is the research and deve...,2014-01-01


In [80]:
generated_description_prompt = """
Tell me about ${company}'s product.

Next, rate your confidence in the description on a scale from 1 (not confident) to 10 (highly confident). Remember to follow the example format!

**EXAMPLES BELOW**

Company: Genmo
Website: https://www.genmo.ai/
Description: Genmo is a free tool that creates videos and images using artificial intelligence. Genmo can also generate creative text formats of text content, like poems, code, scripts, musical pieces, email, letters, etc.

Company: OnePane
Website: https://www.onepane.ai/
Description: Onepane is a company that offers an AI companion for enhanced DevOps & SRE efficiency. Onepane offers a GenAI solution providing unparalleled unified insights and control over your Cloud resources. Onepane helps with root cause analysis, cloud governance, and optimization strategies.

**YOUR TURN**

Company: $company
Website: $website
Current Product Description: $description
________________________
Description (in two sentences):
Confidence Interval:
Reasoning:
________________________
"""



In [81]:
filter_prompt = """
On a scale of 1-10, Is the goal of the product of $company ($website) to be used by a task & occupation in the US workforce? 

For each startup, provide
    + a specific rating (1-10) of how likely tool would be to replace part of a human workforce activity. 
    + If rating > 5, give ONET job title that the tool is automating.

**EXAMPLES BELOW**
Description: DecoverAI is an advanced research and eDiscovery platform tailored specifically for legal teams. It leverages cutting-edge AI technology to streamline and accelerate the discovery process, enabling lawyers to uncover critical insights and evidence efficiently.
Rating: 10
Reasoning (one sentence): Replacing research task for paralegal
ONET Task: Paralegal

Description: Flowfile is an AI-powered platform designed to streamline and automate repetitive tasks for real estate teams. Its features include document automation, lead qualification, and data management, freeing up valuable time for agents to focus on high-impact activities.
Rating: 10
Reasoning (one sentence): Replacing tedious tasks for real estate agents
ONET Task: Real Estate Agent

Description: AI Mavericks hosts high-profile events centered around AI and business growth, targeting founders, CEOs, and leadership teams seeking to leverage AI for strategic advantage. Their events provide an immersive platform for learning, networking, and gaining practical insights on integrating AI into business strategies.
Rating: 2
Reasoning (one sentence): Not replacing an occupation or task, just hosting events
ONET Task: N/A

**YOUR TURN**
Description of $company: $description
________________________
Rating: --
Reasoning: --
________________________
"""

In [82]:
parsed_description_prompt = """
Your role is to describe the jobs/tasks, industries, and customers that $company is targeting. Then, provide a confidence interval (1-10) on how sure you are about the response. 

For each startup, describe:
+ Tasks/Jobs being automated
+ Industry that tasks/jobs belong to
+ specific customer using the tool. who is using the tool?
    - DO NOT INCLUDE AN EXPLANATION

Remember to follow the example format!

**EXAMPLES BELOW**

Company: Petville
Website: petville.co/pricing/biz
Description: Petville Global is a B2B CRM SaaS platform that utilizes advanced technologies like AI/ML and neural net to streamline and expand operations for pet businesses and veterinary clinics both locally and globally. The platform offers deep data analytics and marketing tools, helping businesses save an average of 22% on CRM and vet tech costs.
Tasks/Jobs: Data analysis, Marketing automation, Appointment scheduling, Inventory management
Industry: Customer management
Customers: Veterinarian
Confidence Interval: 8
Reasoning: The description aligns well with the company's focus on CRM for pet-related businesses and includes specific tasks and industries that match the provided information.

Company: Thunder
Website: thundercompute.com
Description: Thunder is a decentralized, peer-to-peer cloud computing platform designed to democratize access to GPUs and address the persistent GPU shortage. It provides a solution for individuals and businesses seeking high-performance computing power, enabling them to leverage unused GPU resources from around the world.
Tasks/Jobs: GPU resource allocation, Access to unused GPUs, Distributed computing tasks
Industry: Cloud Computing
Customers: Software Developers
Confidence Interval: 9
Reasoning: The platform's focus on GPU allocation and decentralized computing is clear, and the target customers and industry are well-defined and specific.

Company: NonprofitsHQ
Website: www.nonprofitshq.com
Description: NonprofitsHQ is a software suite designed for nonprofits that utilizes AI to automate tasks, manage operations, and improve efficiency, ultimately saving organizations time and resources.
Tasks/Jobs: Fundraising management, Donor relationship management, Grant writing, Event planning
Industry: Non-profit management
Customers: Non-profit employee
Confidence Interval: 7
Reasoning: The description covers essential aspects of nonprofit management and identifies specific tasks and customers, but there could be more detail on the AI's role in these tasks.

**YOUR TURN**

Company: $company
Website: $website
Current Product Description: $description
________________________
Tasks/Jobs: (comma separated list of 4, short)
Industry: 
Customers: (comma separated list)
Confidence Interval: 
Reasoning: 
________________________
"""


In [83]:
#(follow ONET task statements for example on how to format a task: https://www.onetcenter.org/dictionary/20.1/excel/task_statements.html)
#    - Task should not use specific details from the context, just the general work activity
# **Here are 10 Example Job Titles and Tasks**
#     ________________
#     Job Title: Engineering teachers, postsecondary
#     Task: Supervise undergraduate or graduate teaching, internship, and research work.
    
#     Job Title: Instructional coordinators
#     Task: Adapt instructional content or delivery methods for different levels or types of learners.
    
#     Job Title: Technical writers
#     Task: Edit, standardize, or make changes to material prepared by other writers or establishment personnel.
    
#     Job Title: Nurse midwives
#     Task: Plan, provide, or evaluate educational programs for nursing staff, health care teams, or the community.
    
#     Job Title: Business teachers, postsecondary
#     Task: Develop and maintain course web sites.
    
#     Job Title: Airline pilots, copilots, and flight engineers
#     Task: Coose routes, altitudes, and speeds that will provide the fastest, safest, and smoothest flights.
    
#     Job Title: Electronic equipment installers and repairers, motor vehicles
#     Task: Confer with customers to determine the nature of malfunctions.
    
#     Job Title: Geography teachers, postsecondary
#     Task: Maintain regularly scheduled office hours to advise and assist students.
    
#     Job Title: Ophthalmologists, except pediatric
#     Task: Document or evaluate patients' medical histories.
    
#     Job Title: Special education teachers, elementary school
#     Task: Collaborate with other teachers or administrators to develop, evaluate, or revise elementary school programs.
#     ________________

In [84]:
examples_prompt = """
Imagine you're explaining the product from $company to someone unfamiliar with it. Provide 3 concise examples of how this product could be used in a real-world scenario. Keep the descriptions general and avoid mentioning the company name.

For each example, describe:
+ Context: description of WHAT the tool is doing
+ Job Title and Task that the tool replaces
    - Job Title should be broad and not based on specific context!
    - Task should be specific to what the tool is automating!
+ Next, rate your confidence in example task on a scale from 1 (not confident) to 10 (highly confident).


**EXAMPLES BELOW**

Company: Blanc
Website: tryblanc.ai
Blanc is a compliance automation platform designed for fintech companies. It helps streamline regulatory compliance processes by providing a centralized hub for managing policies, monitoring activities, and generating reports.
________________
Context 1: A compliance officer at a fintech company uses Blanc to automate the process of creating and updating compliance policies, ensuring all documents are current and accessible to relevant team members.
Task 1: Verify that all firm and regulatory policies and procedures have been documented, implemented, and communicated.
Job Title 1: Compliance Officers

Context 2: A fintech company uses Blanc to generate automated compliance reports for regulatory audits, ensuring all necessary documentation is readily available and organized.
Task 2: Prepare reports of activities, evaluations, recommendations, or decisions.
Job Title 2: Compliance Officers

Context 3: A fintech company uses Blanc to monitor real-time transactions for potential compliance violations, triggering alerts and generating reports for further investigation.
Task 3: Identify compliance issues that require follow-up or investigation.
Job Title 3: Compliance Officers
________________

Company: Aether
Website: aetherenergie.com/
Aether Energy is an AI-driven platform designed to simplify the process of rooftop solar installation for businesses, providing comprehensive support from project planning and financing to installation and ongoing maintenance. This platform aims to streamline and optimize the entire solar energy journey for installers. 
________________
Context 1: A solar installer uses Aether to quickly create detailed project plans for rooftop solar installations, including system size, panel placement, and wiring diagrams. 
Task 1: Diagram layouts and locations for photovoltaic (PV) arrays and equipment, including existing building or site features.
Job Title 1: Solar Photovoltaic Installers

Context 2: A business owner leverages Aether to secure financing for their rooftop solar project, providing them with customized loan options and streamlined application processes. 
Task 2: Prepare solar installation project proposals, quotes, budgets, or schedules.
Job Title 2: Solar Photovoltaic Installers

Context 3: A solar installer uses Aether to manage the installation process, tracking materials, scheduling technicians, and coordinating with subcontractors, ensuring smooth project execution. 
Task 3:  Monitor work of contractors and subcontractors to ensure projects conform to plans, specifications, schedules, or budgets.
Job Title 3: Solar Energy Installation Managers
________________


Remember to follow the example format! Please include numbers!

**YOUR TURN**
Company:$company
Website: $website
Current Description: $description
$parsed_description
________________
**Example 1**
Context 1: 
Job Title 1: 
Task 1:
Confidence Interval 1:
Reasoning 1:

**Example 2**
Context 2: 
Job Title 2: 
Task 2:
Confidence Interval 2:
Reasoning 2:

**Example 3**
Context 3:
Job Title 3: 
Task 3:
Confidence Interval 3:
Reasoning 3:
________________
"""

In [124]:
# Apply nest_asyncio
nest_asyncio.apply()


class prompting():
    def __init__(self):
        self.results_df = pd.DataFrame(columns=["organization name", "value"])

    def set_current_results_df(self, results_df):
        self.results_df = results_df

    async def iterate(self, df, prompt_template, args, value, batch_size=10, start=0, end=False):
        if end == False:
            end = len(df)
        self.results_df = pd.DataFrame(columns=["organization name", value])
        if value in list(df.columns):
            if start != 0:
                self.results_df = pd.concat([df[["organization name", value]].iloc[:start], self.results_df], axis=0)
            df = df.drop(columns=[value])

        batch_prompts = []
        batch_indices = []

        for i, row in list(df.iterrows())[start:end]:
            name = row['organization name']
            website = row['website']
            prompt = prompt_template

            for arg in args:
                prompt = prompt.replace(f"${arg[0]}", row[arg[1]])

            batch_prompts.append((i, name, prompt))
            batch_indices.append(i)

            if len(batch_prompts) >= batch_size:
                await self.process_batch(batch_prompts)
                batch_prompts = []

        if batch_prompts:
            await self.process_batch(batch_prompts)
            self.results_df.to_csv("../output/current_results_df_prompting.csv")

        df = df.merge(self.results_df, on='organization name', how='left')
        return df

    async def process_batch(self, batch_prompts):
        async with aiohttp.ClientSession() as session:
            tasks = []
            for i, name, prompt in batch_prompts:
                tasks.append(self.fetch_result(session, i, name, prompt))
            await asyncio.gather(*tasks)

    async def fetch_result(self, session, i, name, prompt):
        failure_count = 0
        while True:
            try:
                print(f"******************************\nProcessing {i}: {name}")
                result = await x_gemini.ask(session, prompt)
                if result == "N/A": break  # explicit material

                text = re.sub(r"#|#\s+|_|\*", "", result).strip()

                self.results_df.loc[i] = [name, text]

                    
                break
            except Exception as e:
                print(failure_count)
                failure_count += 1
                if failure_count > 10:
                    break
                print(f"Error processing {i}, {name}: {e}")
                await asyncio.sleep(20)

prompting_class = prompting()


In [125]:
len(df)

900

## Generated Description

In [126]:
args = [["company","organization name"], ["website","website"], ["description","description_all"]]

async def main(df):
    df = await prompting_class.iterate(df, generated_description_prompt, args, "generated_description_llm")
    df.to_csv('../output/df_with_generated_description.csv', index=False)
    return df

df = asyncio.run(main(df))

******************************
Processing 0: AI Mavericks
******************************
Processing 1: Campaign Brain
******************************
Processing 2: Sentius
******************************
Processing 3: Vadar Alpha
******************************
Processing 4: Cognifyze
******************************
Processing 5: Beneree
******************************
Processing 6: Insightful Revenue
******************************
Processing 7: Sups
******************************
Processing 8: SQDL
******************************
Processing 9: Intelizen
******************************
Processing 10: Panel AI
******************************
Processing 11: Tely AI
******************************
Processing 12: Telow
******************************
Processing 13: crear.ai
******************************
Processing 14: Entwyne
******************************
Processing 15: Vectari
******************************
Processing 16: Post Cheetah
******************************
Processing 17: Argo Labs
******

In [127]:
cols = ["generated_description", "generated_description_conf_interval", "generated_description_conf_interval_reasoning"]

def extract_data(text):
    if pd.isnull(text): return {}
    text = text.replace("\n"," ").replace("  "," ").replace("*","").replace("/10","").replace("_","").replace("#","")
    text = re.sub(r'\s?\([^)]*\)', '', text)
    pattern = r"^.*?\s*Description:?\s*(.*)Confidence Interval:\s*(\d+)\s*Reasoning:\s*(.*)$"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        description = match.group(1).strip()
        confidence_interval = match.group(2).strip()
        reasoning = match.group(3).strip()
        
        result = {
            "generated_description": description,
            "generated_description_conf_interval": confidence_interval,
            "generated_description_conf_interval_reasoning": reasoning
        }
        return result
        
    else:
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["generated_description_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)
len(df)

FAILURE TO MATCH


902

In [128]:
# df = pd.read_csv("../output/df_with_generated_description.csv", index_col=0).reset_index(drop=True)
# len(df)

In [129]:
df = df.dropna(subset=["generated_description"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

898

## Filter Prompt

In [130]:
# args = [["company","organization name"], ["website","website"], ["description","description_all"]]

# async def main(df):
#     df = await prompting_class.iterate(df, filter_prompt, args, "automation_rating_llm")
#     df.to_csv('../output/df_with_filter.csv', index=False)
#     return df

# df = asyncio.run(main(df))

In [131]:
# cols = ["automation_rating", "automation_rating_reasoning"]

# def extract_data(text):
#     if pd.isnull(text): return {}
#     text = re.sub(r'\s+', ' ', text).replace("\n"," ").replace("  "," ").replace("*","").replace("/10","").replace("_","").replace("#","")
#     pattern = r"Rating: (\d+)\s+Reasoning:\s+(.*)"

#     # Search for the pattern in the text
#     match = re.search(pattern, text, re.DOTALL)
#     if match:
#         automation_rating = match.group(1).strip()
#         automation_rating_reasoning = match.group(2).strip()

#         result = {
#             "automation_rating": automation_rating,
#             "automation_rating_reasoning": automation_rating_reasoning
#         }
#         return result

#     else:
#         print(text)
#         print("FAILURE TO MATCH")
#         return {}
        
# for col in cols:
#     if col in list(df.columns): df=df.drop(columns=[col])
        
# results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["automation_rating_llm"]),axis=1)))
# df = pd.concat([df, results_df],axis=1)


In [132]:
# df = df.dropna(subset=["automation_rating"],axis=0)
# df = df.drop_duplicates(subset=['organization name'], keep='first')
# df = df.reset_index(drop=True)

In [133]:
# for x in range(len(df))[:1]:
#     if int(df.automation_rating[x]) <= 10:
#         print(df.generated_description[x])
#         print(df.automation_rating_llm[x])

## Parsed Description

In [134]:
cols = ["parsed_description", "parsed_description_conf_interval", "parsed_description_conf_interval_reasoning", "Tasks/Jobs","Industry","Customers"]  
args = [["company","organization name"],["website","website"], ["description","generated_description"]]

async def main(df):
    df = await prompting_class.iterate(df, parsed_description_prompt, args, "parsed_description_llm")
    df.to_csv('../output/df_with_parsed_description.csv', index=False)
    return df

df = asyncio.run(main(df))




******************************
Processing 0: AI Mavericks
******************************
Processing 1: Campaign Brain
******************************
Processing 2: Sentius
******************************
Processing 3: Vadar Alpha
******************************
Processing 4: Cognifyze
******************************
Processing 5: Beneree
******************************
Processing 6: Insightful Revenue
******************************
Processing 7: Sups
******************************
Processing 8: SQDL
******************************
Processing 9: Intelizen
******************************
Processing 10: Panel AI
******************************
Processing 11: Tely AI
******************************
Processing 12: Telow
******************************
Processing 13: crear.ai
******************************
Processing 14: Entwyne
******************************
Processing 15: Vectari
******************************
Processing 16: Post Cheetah
******************************
Processing 17: Argo Labs
******

In [135]:
def extract_data(text):
    if pd.isnull(text): return {}
    text = re.sub(r'\s+', ' ', text)  
    text = text.replace("_", "").replace("*", "").replace("#", "")
    text = re.sub(r'\s?\([^)]*\)', '', text)
    pattern = r".*?Tasks/Jobs:\s*(.*?)\s*Industry:\s*(.*?)\s*Customers:\s*(.*?)\s*Confidence Interval:\s*(.*?)\s*Reasoning:\s*(.*)"
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        tasks_jobs = match.group(1).strip()
        industry = match.group(2).strip()
        customers = match.group(3).strip()
        confidence_interval = match.group(4).strip()
        reasoning = match.group(5).strip()
        
        result = {
            "parsed_description": 
            "Tasks/Jobs: " + tasks_jobs + "\n" + "Industry: " + industry + "\n" + "Customers: "+ customers,
            "Tasks/Jobs": tasks_jobs,
            "Industry": industry,
            "Customers": customers,
            "parsed_description_conf_interval": confidence_interval,
            "parsed_description_conf_interval_reasoning": reasoning
        }
        
        return result
    else:
        print(text)
        print("FAILURE TO MATCH")
        return {}
        
for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])
        
results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["parsed_description_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


In [136]:
len(df)

898

In [137]:
df = df.dropna(subset=["parsed_description"],axis=0)
df = df.drop_duplicates(subset=['organization name'], keep='first')
df = df.reset_index(drop=True)
len(df)

898

In [138]:
df.to_csv('../output/results/df_with_parsed_description.csv', index=False)

In [140]:
# df["parsed_description"].values

## Examples

In [127]:
cols = ["situation1", "situation1_conf_interval", "situation1_conf_interval_reasoning", "situation2", "situation2_conf_interval", "situation2_conf_interval_reasoning","situation3", "situation3_conf_interval", "situation3_conf_interval_reasoning",'Example1','Job1','Job1_title','Example2','Job2','Job2_title','Example3','Job3','Job3_title']
args = [["company","organization name"],["website","website"], ["description","generated_description"], ["parsed_description","parsed_description"]]


async def main(df):
    df = await prompting_class.iterate(df, examples_prompt, args, "examples_llm",1060)
    df.to_csv('../output/df_with_examples.csv', index=False)
    return df

df = asyncio.run(main(df))


******************************
Processing 0: Anima Biotech
******************************
Processing 1: Few - Digital Product Agency
******************************
Processing 2: Bliss Point
******************************
Processing 3: Sober Grid
******************************
Processing 4: Trupik
******************************
Processing 5: Seamless Planet
******************************
Processing 6: Quartet Health
******************************
Processing 7: ON
******************************
Processing 8: ROBOTERRA, Inc
******************************
Processing 9: Esperanto Technologies


In [128]:

def extract_data(text):
    if pd.isnull(text): return {}
    # Normalize the text to ensure consistent whitespace and remove unwanted characters.
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace("*", "").replace("/10", "").replace("_", "").replace("#", "").replace("- ","")
    text = re.sub(r'\(.*?\)', '', text)
    vals = {}
    i = 1
    while i <=3:
        # Regex pattern adjusted to handle fractions in confidence intervals like '9/10'
        pattern = rf"""
        Context\s+{i}:\s+(.*?)\s+
        Job Title\s+{i}:\s+(.*?)\s+
        Task\s+{i}:\s+(.*?)\s+
        Confidence\s+Interval\s+{i}:\s+(\d+(?:/\d+)?)\s+
        Reasoning\s+{i}:\s+(.*?)(?=\s*Context\s+{i + 1}:|$)"""
        pattern = pattern.replace("        ","").replace("\n","")
        match = re.search(pattern, text, re.DOTALL)
        if not match:
            print(text+"\n\n")
            print(f"No matches found for Example {i}")  # Debug if no examples are found
            return {}
            break

        example_text, job_title, task, confidence_interval, reasoning = match.groups()
        description = job_title + " that " + task.lower()
        example_key = f"Example{i}"
        vals[example_key] = example_text.strip()
        vals[f"Example_Task{i}"] = task.strip()
        vals[f"Example_Job_Title{i}"] = job_title.strip()
        vals[f"Example_Description{i}"] = description.strip()
        vals[f"situation{i}_conf_interval"] = confidence_interval.strip()
        vals[f"situation{i}_conf_interval_reasoning"] = reasoning.strip()

        i += 1  # Prepare to search for the next example
    return vals

for col in cols:
    if col in list(df.columns): df=df.drop(columns=[col])

results_df = pd.DataFrame(list(df.apply(lambda x: extract_data(x["examples_llm"]),axis=1)))
df = pd.concat([df, results_df],axis=1)


In [129]:
df.to_csv('../output/df_with_examples.csv', index=False)

In [ ]:
#RERUN STOPPED IN MIDDLE
# results_df = pd.read_csv("result.csv",index_col=0)
# df = pd.read_csv('../output/df_with_parsed_description.csv',index_col=0)
# prompting_class.set_current_results_df(results_df)
# df = df.merge(results_df, on='organization name', how='left')

In [ ]:
# df = pd.read_csv('../output/results/df_with_examples.csv',index_col=0)


In [130]:
df.iloc[2].values

array(['Bliss Point', '2014-01-01', '101-250',
       'Advertising,Machine Learning,Marketing,Video Streaming',
       'Bliss Point renders video advertising and performance marketing services for streaming and television using machine learning and algorithms.',
       'Bliss Point renders video advertising and performance marketing services for streaming and television using machine learning and algorithms.',
       'www.blisspointmedia.com/',
       'Bliss Point renders video advertising and performance marketing services for streaming and television using machine learning and algorithms. Bliss Point renders video advertising and performance marketing services for streaming and television using machine learning and algorithms.',
       'Description (in two sentences): Bliss Point provides video advertising and performance marketing solutions for streaming and television platforms. They leverage machine learning and advanced algorithms to optimize campaigns and deliver measurable resu